In [203]:
import random
import pandas as pd
import numpy as np

In [204]:
words = pd.read_csv("exp_files/condition_files/words4exp.csv")

# Generating Word Lists
In this block we want to generate a dictionary of lists that correspond with the word categories in the `words4exp.csv` file. This will allow us to sample a subset of the words from each category to show in the AMP and then use the words that were **not** in the AMP for the lexical decision making task. This code will be pasted into PsychoPy in order to generate the word lists for each participant.

- 70 moral words (14 from each foundation), 10 nonword primes, 2 control words, and 82 nonword targets will go in the AMP.
- 50 moral words, 50 neutral words, and 100 nonwords will go into the LDT

First, we will divide up the dataframe based on word category, sorting words from each category into separate dictionary keys in a dictionary we'll call `unique_cats`.

In [205]:
unique_cats = {k:None for k in words.category.unique()}
for k,v in unique_cats.items():
    unique_cats[k] = list(words[words.category == k].index.values)

Now we will draw a random sample of 7 words per MFD category, 92 nonwords (82 targets and 10 primes), and the 2 controls for the AMP. What's left we'll put into the LDT. 

In [206]:
full_amp_words = {k:None for k in unique_cats.keys() if k != "neutral"}
full_ldt_words = {k:None for k in unique_cats.keys()}
for k,v in unique_cats.items():
    if "." in str(k): 
        full_amp_words[k] = random.sample(unique_cats[k],7)
        full_ldt_words[k] = [x for x in unique_cats[k] if x not in full_amp_words[k]]
    if k == "nonword":
        full_amp_words[k] = random.sample(unique_cats[k],92)
        full_ldt_words[k] = [x for x in unique_cats[k] if x not in full_amp_words[k]]
    if k =="control":
        full_amp_words[k] = unique_cats[k]
    if k == "neutral":
        full_ldt_words[k] = unique_cats[k]

## Generating the final list of words for the AMP

In [207]:
amp_moral_words = []
amp_nonwords = []
for k,v in full_amp_words.items():
    if k != "nonword":
        for i in v:
            amp_moral_words.append(i)
    if k == "nonword":
        for i in v:
            amp_nonwords.append(i)
            
# Taking a random sample of 10 of the nonwords to use as primes, and then adding them to the primes list
amp_nonword_primes = random.sample(amp_nonwords,10)
amp_primes_list = amp_moral_words + amp_nonword_primes

for x in amp_nonword_primes:
  amp_nonwords.remove(x)

In [208]:
#We should have 82 total matched pairs in the amp.
len(amp_full_list)

82

In [209]:
# Pulling the words, correct answers, and categories for all of the indices for our moral words plus our ten nonword controls
AMP_wordlist = words.loc[amp_primes_list].words.values
AMP_corr_ans = words.loc[amp_primes_list].correct_amp.values
AMP_category = words.loc[amp_primes_list].category.values

#pulling the nonwords and masks for our targets that weren't selected to be included in the prime
AMP_nonwordlist = words.loc[amp_nonwords].words.values
AMP_masklist = words.loc[amp_nonwords].masks.values

In [210]:
AMP_nonwordlist

array(['rivernoat', 'laming', 'totalestic', 'chared', 'shoirmaster',
       'squets', 'carvels', 'neutralust', 'tabin', 'sgrawl', 'imebreaker',
       'ablow', 'tenuflect', 'gazily', 'chreams', 'quarterbocks',
       'halaise', 'proached', 'blunderints', 'gedious', 'vicissifude',
       'ebbitter', 'pharing', 'brotesque', 'synchronazers', 'pumber',
       'nonobservalt', 'esquisiteness', 'gondensed', 'menizen', 'hatigue',
       'presentkess', 'pulge', 'feleological', 'focomotive', 'grovelire',
       'nackers', 'nomaric', 'troubla', 'clattening', 'Oregoniuns',
       'erks', 'nomparisons', 'pistraught', 'simplustic', 'fostet',
       'thabbily', 'abointment', 'ecgagement', 'infet', 'nuckheap',
       'openet', 'gandbrake', 'gournful', 'oam', 'letish', 'ning',
       'ittutable', 'reprisols', 'sliptery', 'godlaness', 'clamorek',
       'alar', 'neputized', 'lachelor', 'pullness', 'fonderland',
       'faspberry', 'gillipede', 'seachwear', 'festate', 'epigrach',
       'framatic', 'supe

Now we will create a couple of dataframes that will contain the word, the correct answer, the nonword, and the mask. We will then shuffle the words and the nonwords so that they aren't in the same order for every participant. Finally, we'll concatenate them together.

In [211]:
# df1 is the words that will be used as primes and their associated categories and correct answers

df1 = pd.DataFrame(
    {'words': AMP_wordlist.tolist(),
     'corr_ans': AMP_corr_ans.tolist(),
     'category': AMP_category.tolist()
    })

# df2 is the nonwords that will be used as targets and their associated masks

df2 = pd.DataFrame(
     {'nonwords': AMP_nonwordlist.tolist(),
     'mask': AMP_masklist.tolist(),
    })

# Shuffling the words once and then generating four 

primes_rep1 = df1.sample(frac=1).reset_index(drop=True)
targets_rep1 = df2.sample(frac=1).reset_index(drop=True)

primes_rep2 = df1.sample(frac=1).reset_index(drop=True)
targets_rep2 = df2.sample(frac=1).reset_index(drop=True)

primes_rep3 = df1.sample(frac=1).reset_index(drop=True)
targets_rep3 = df2.sample(frac=1).reset_index(drop=True)

primes_rep4 = df1.sample(frac=1).reset_index(drop=True)
targets_rep4 = df2.sample(frac=1).reset_index(drop=True)

# Concatenating all of the primes and targets horizontally

rep1 = pd.concat([primes_rep1,targets_rep1], axis = 1)
rep2 = pd.concat([primes_rep2,targets_rep2], axis = 1)
rep3 = pd.concat([primes_rep3,targets_rep3], axis = 1)
rep4 = pd.concat([primes_rep4,targets_rep4], axis = 1)

# Concatenating all of the reps horizontally

df_full = pd.concat([rep1,rep2,rep3,rep4], axis = 0)

Finally, we will write this to a CSV that will be used for this participant and then deleted.

In [212]:
df_full.reset_index(inplace=True, drop=True)
df_full

,words,corr_ans,category,nonwords,masks
0,encourage,left,authority.virtue,chadows,&&&&&&&
1,powerful,left,authority.virtue,infet,&&&&&
2,safe,left,care.virtue,menizen,&&&&&&&
3,ejanulated,NaN,nonword,festate,&&&&&&&
4,officers,right,authority.vice,gillipede,&&&&&&&&&
5,slavery,right,loyalty.vice,clattening,&&&&&&&&&&
6,arrests,right,authority.vice,fonderland,&&&&&&&&&&
7,attacks,right,care.vice,sickyay,&&&&&&&
8,threatened,right,fairness.vice,laming,&&&&&&
9,praise,left,authority.virtue,nomparisons,&&&&&&&&&&&


In [213]:
df_full.to_csv("exp_files/condition_files/AMP_conditions.csv")

## Lexical Decision Task

In the Lexical Decision Making tasks (LDT) we will have LDT 50 moral words, 50 neutral words, 100 nonwords

In [214]:
ldt_wordlist = []
for k,v in full_ldt_words.items():
    if k != "control":
        for i in v:
            ldt_wordlist.append(i)
            
# Pulling the words, correct answers, categories, & masks for all of the indices for our words
LDT_words = words.loc[ldt_wordlist].words.values
LDT_corr = words.loc[ldt_wordlist].correct_ldt.values
LDT_category = words.loc[ldt_wordlist].category.values
LDT_masklist = words.loc[ldt_wordlist].masks.values
            
df_ldt = pd.DataFrame(
    {'words': LDT_words.tolist(),
     'corr_ans': LDT_corr.tolist(),
     'category': LDT_category.tolist(),
     'mask': LDT_masklist.tolist()
    })

In [215]:
# We should have 200 LDT words
df_ldt

,words,corr_ans,category,mask
0,care,left,care.virtue,&&&&
1,health,left,care.virtue,&&&&&&
2,compassion,left,care.virtue,&&&&&&&&&&
3,improve,left,care.virtue,&&&&&&&
4,restore,left,care.virtue,&&&&&&&
5,death,left,care.vice,&&&&&
6,violence,left,care.vice,&&&&&&&&
7,died,left,care.vice,&&&&
8,torture,left,care.vice,&&&&&&&
9,victim,left,care.vice,&&&&&&


In [217]:
df_ldt.to_csv("exp_files/condition_files/LDT_conditions.csv")